# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
import pandas as pd
from bs4 import BeautifulSoup
from scipy.stats import pearsonr


In [35]:
def clear_data(string1):
    """Removes citation tags like [note 1] from strings."""
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')

def get_area(team):
    """Maps NHL teams to their metropolitan areas based on the nhl_cities index."""
    for each in list(nhl_cities.index.values):
        if team in each:
            return nhl_cities.at[each, 'Metropolitan area']

In [37]:
# Load the relevant table from the HTML file
cities = pd.read_html("assets/wikipedia_data.html")[1]

# Select only relevant columns
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))

# Set up a DataFrame to map NHL teams to metropolitan areas
nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
nhl_cities = nhl_cities.drop(['—', ''], axis=0)


In [39]:
# Load the NHL data and filter for 2018
nhl_df = pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0)  # Only 2018 data

# Clean the team names by removing trailing asterisks and whitespace
nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())

# Map team names to metropolitan areas
nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))


In [40]:
# Calculate win/loss ratio per area
out = []
for group, frame in nhl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W'], errors='coerce'))
    total_losses = np.sum(pd.to_numeric(frame['L'], errors='coerce'))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches if total_matches > 0 else np.nan
    out.append({'Area': group, 'Ratio': ratio})

# Convert to DataFrame
win_loss_df = pd.DataFrame(out).set_index('Area')


In [43]:
from scipy.stats import pearsonr
import pandas as pd
import numpy as np
import re

def nhl_correlation():
    # Load and clean Wikipedia data
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    cities['NHL'] = cities['NHL'].apply(lambda x: clear_data(x))
    
    # Map NHL teams to metropolitan areas
    nhl_cities = cities[['Metropolitan area', 'NHL']].set_index('NHL')
    nhl_cities = nhl_cities.drop(['—', ''], axis=0)
    
    # Load NHL data and filter for 2018
    nhl_df = pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df['year'] == 2018].drop([0, 9, 18, 26], axis=0)
    nhl_df['team'] = nhl_df['team'].apply(lambda x: x[:-1].strip() if x.endswith("*") else x.strip())
    
    # Map team names to metropolitan areas
    nhl_df['area'] = nhl_df['team'].apply(lambda x: x.split(" ")[-1])
    nhl_df['area'] = nhl_df['area'].apply(lambda x: get_area(x))
    
    # Calculate win/loss ratio per area
    out = []
    for group, frame in nhl_df.groupby('area'):
        total_wins = np.sum(pd.to_numeric(frame['W'], errors='coerce'))
        total_losses = np.sum(pd.to_numeric(frame['L'], errors='coerce'))
        total_matches = total_wins + total_losses
        ratio = total_wins / total_matches if total_matches > 0 else np.nan
        out.append({'Area': group, 'Ratio': ratio})
    
    # Create DataFrame for win/loss ratio
    win_loss_df = pd.DataFrame(out).set_index('Area')
    
    # Prepare the population data
    population = cities[['Metropolitan area', 'Population (2016 est.)[8]']].set_index('Metropolitan area')
    population['Population (2016 est.)[8]'] = pd.to_numeric(population['Population (2016 est.)[8]'], errors='coerce')
    
    # Merge win/loss ratio data with population data
    out_df = pd.merge(win_loss_df, population, how="inner", left_index=True, right_index=True)
    
    # Extract population and win/loss ratios for the correlation calculation
    population_by_region = out_df['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = out_df['Ratio'].tolist()
    
    # Assertions to ensure correct list lengths
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    # Calculate Pearson correlation
    correlation, _ = pearsonr(population_by_region, win_loss_by_region)
    
    return correlation


In [44]:
nhl_correlation()

0.01230899645574425

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nba_correlation():
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    nba_2018 = nba_df[(nba_df['year'] == 2018)]
    nba_2018 = nba_2018[['team', 'W', 'L']]

    for i in nba_2018.index:  
              nba_2018.at[i, 'W/L'] = int((nba_2018.at[i, 'W'])) / (int(nba_2018.at[i, 'W']) + int(nba_2018.at[i, 'L']))  

    for i in nba_2018.index:
          nba_2018.at[i, 'team'] = nba_2018.at[i, 'team'].replace("*", "")
          nba_2018.at[i, 'team'] = nba_2018.at[i, 'team'].replace("(", "")
          nba_2018.at[i, 'team'] = nba_2018.at[i, 'team'].replace(")", "")
          nba_2018.at[i, 'team'] = nba_2018.at[i, 'team'].replace(u"\xa0", u"")
          nba_2018.at[i, 'team'] = nba_2018.at[i, 'team'].rstrip() 
          nba_2018.at[i, 'team'] = ''.join([i for i in nba_2018.at[i, 'team'] if not i.isdigit()])

    nba_2018['city'] = nba_2018['team']
    nba_2018['city'] = nba_2018['city'].map({'Toronto Raptors':'Toronto',
         'Boston Celtics':'Boston',
         'Philadelphia ers':'Philadelphia',
         'Cleveland Cavaliers':'Cleveland',
         'Indiana Pacers':'Indianapolis',
         'Miami Heat':'Miami–Fort Lauderdale',
         'Milwaukee Bucks':'Milwaukee',
         'Washington Wizards':'Washington, D.C.',
         'Detroit Pistons':'Detroit',
         'Charlotte Hornets':'Charlotte',
         'New York Knicks':'New York City',
         'Brooklyn Nets':'New York City',
         'Chicago Bulls':'Chicago',
         'Orlando Magic':'Orlando',
         'Atlanta Hawks':'Atlanta',
         'Houston Rockets':'Houston',
         'Golden State Warriors':'San Francisco Bay Area',
         'Portland Trail Blazers':'Portland',
         'Oklahoma City Thunder':'Oklahoma City',
         'Utah Jazz':'Salt Lake City',
         'New Orleans Pelicans':'New Orleans',
         'San Antonio Spurs':'San Antonio',
         'Minnesota Timberwolves':'Minneapolis–Saint Paul',
         'Denver Nuggets':'Denver',
         'Los Angeles Clippers':'Los Angeles',
         'Los Angeles Lakers':'Los Angeles',
         'Sacramento Kings':'Sacramento',
         'Dallas Mavericks':'Dallas–Fort Worth',
         'Memphis Grizzlies':'Memphis',
         'Phoenix Suns':'Phoenix'})

    cities = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
    cities.set_index('Metropolitan area', inplace = True)
    nba_2018.set_index("city", inplace = True)
    nba_2018_final = nba_2018.join(cities, how = 'left')
    nba_2018_final = nba_2018_final[['W/L', 'Population (2016 est.)[8]']]

    nba_2018_final['W/L'] = nba_2018_final['W/L'].apply(lambda x : float(x))
    nba_2018_final['Population (2016 est.)[8]'] = nba_2018_final['Population (2016 est.)[8]'].apply(lambda x : float(x))

    repeat= []
    for i in range(0,  nba_2018_final.shape[0]-1): 
          if ((nba_2018_final.index[i] == nba_2018_final.index[i+1]) and (nba_2018_final.index[i] not in repeat)):
                repeat.append(nba_2018_final.index[i])

    for i in repeat:
          nba_2018_final.at[i, 'W/L'] = np.average(nba_2018_final.at[i, 'W/L'])

    nba_2018_final = nba_2018_final[~nba_2018_final.index.duplicated(keep='first')]
    
    
    population_by_region = nba_2018_final['Population (2016 est.)[8]'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region   = nba_2018_final['W/L'].tolist() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


In [13]:
nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def get_area(team):
    for each in list(mlb_cities.index.values):
        if team in each:
            return mlb_cities.at[each, 'Metropolitan area']

def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x))
mlb_cities = cities[['Metropolitan area', 'MLB']].set_index('MLB')
mlb_cities = mlb_cities.drop(['—', ''], axis=0)
mlb_df = mlb_df[mlb_df['year'] == 2018]  # get only 2018 stats no need of dropping rows
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
mlb_df['area'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])
mlb_df['area'] = mlb_df['area'].apply(lambda x: get_area(x))
mlb_df.at[0, 'area'] = 'Boston'
out = []
for group, frame in mlb_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])


def mlb_correlation(): 
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_mlb_data():
    return out_df

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [16]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clear_data(string1):
    if re.search(r'\[[a-z]* [0-9]+\]', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\[[a-z]* [0-9]+\]', string1).group(), '')


def clear_nba_data(string1):
    if re.search(r'\*|\+', string1) is None:
        return string1
    else:
        return string1.replace(re.search(r'\*|\+', string1).group(), '')


def get_area(team):
    for each in list(nfl_cities.index.values):
        if team in each:
            return nfl_cities.at[each, 'Metropolitan area']
        


cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities['NFL'] = cities['NFL'].apply(lambda x: clear_data(x))
nfl_cities = cities[['Metropolitan area', 'NFL']].set_index('NFL')
nfl_cities = nfl_cities.drop(['—', ''], axis=0)
nfl_df = pd.read_csv("assets/nfl.csv")
nfl_df = nfl_df[nfl_df['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35])  # get only 2018 stats
population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
population = population.set_index('Metropolitan area')
nfl_df['team'] = nfl_df['team'].apply(lambda x: clear_nba_data(x))
nfl_df['area'] = nfl_df['team'].apply(lambda x: x.split(" ")[-1])
nfl_df['area'] = nfl_df['area'].apply(lambda x: get_area(x))
out = []
for group, frame in nfl_df.groupby('area'):
    total_wins = np.sum(pd.to_numeric(frame['W']))
    total_losses = np.sum(pd.to_numeric(frame['L']))
    total_matches = total_wins + total_losses
    ratio = total_wins / total_matches
    out_dict = {
        'Area': group,
        'Ratio': ratio
    }
    out.append(out_dict)
new_df = pd.DataFrame(out)
new_df = new_df.set_index('Area')
out_df = pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
out_df['Population (2016 est.)[8]'] = pd.to_numeric(out_df['Population (2016 est.)[8]'])

def nfl_correlation(): 
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    population_by_region = out_df['Population (2016 est.)[8]'].to_list()
    win_loss_by_region = out_df['Ratio'].to_list()

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return np.float64(stats.pearsonr(population_by_region, win_loss_by_region)[0])

def get_nfl_data():
    return out_df

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")


def nhl_df():
    Big4='NHL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    
    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'\*',"")
    _df = _df[['team','W','L']]

    dropList=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W'] and row['L']==row['W']:
            dropList.append(i)
    _df=_df.drop(dropList)

    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W': int, 'L': int})
    _df['W/L%'] = _df['W']/(_df['W']+_df['L'])
    
    merge=pd.merge(team,_df,'inner', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})  

    return merge[['W/L%']]



def nba_df():
    Big4='NBA'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')

    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W/L%']]
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})
    return merge[['W/L%']]



def mlb_df(): 
    Big4='MLB'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('\ Sox','Sox')
    team['team']=team['team'].str.replace('[\w.]*\ ','')

    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W-L%']]
    _df.rename(columns={"W-L%": "W/L%"},inplace=True)
    _df['team']=_df['team'].str.replace('\ Sox','Sox')
    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]

def nfl_df(): 
    Big4='NFL'
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    
    _df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    _df = _df[_df['year'] == 2018]
    _df['team'] = _df['team'].str.replace(r'[\*]',"")
    _df['team'] = _df['team'].str.replace(r'\(\d*\)',"")
    _df['team'] = _df['team'].str.replace(r'[\xa0]',"")
    _df = _df[['team','W-L%']]
    _df.rename(columns={"W-L%": "W/L%"},inplace=True)
    dropList=[]
    for i in range(_df.shape[0]):
        row=_df.iloc[i]
        if row['team']==row['W/L%']:
            dropList.append(i)
    _df=_df.drop(dropList)

    _df['team'] = _df['team'].str.replace('[\w.]* ','')
    _df['team'] = _df['team'].str.replace('+','')
    _df = _df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,_df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    return merge[['W/L%']]

def create_df(sport):
    if sport =='NFL':
        return nfl_df()
    elif sport =='NBA':
        return nba_df()
    elif sport =='NHL':
        return nhl_df()
    elif sport =='MLB':
        return mlb_df()
    else:
        print("ERROR with intput!")


def sports_team_performance():
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for i in sports:
        for j in sports:
            if i !=j :
                merge=pd.merge(create_df(i),create_df(j),'inner', on = ['Metropolitan area'])
                p_values.loc[i, j]=stats.ttest_rel(merge['W/L%_x'],merge['W/L%_y'])[1]

    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values

In [19]:
sports_team_performance()

AssertionError: The MLB-NFL p-value should be around 0.80